<a href="https://colab.research.google.com/github/kimmunsu/24_langchain_study_colab/blob/main/Ch07_01_arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLM 관련 신규 문서들이 Arxiv 를 통해서 많이 발표되고 있고, 해당 내용들을 자동화하고 LLM을 통해 요약하는 로직을 만들기 위해서는 Langchain 에서 만들어놓은 라이브러리를 사용하는게 편합니다.

lanchain document : https://python.langchain.com/v0.1/docs/integrations/document_loaders/arxiv/

arxiv 라는 것은 Arxiv는 물리학, 수학, 컴퓨터 과학, 정량 생물학, 정량 금융, 통계학, 전기 공학 및 시스템 과학, 경제학 분야의 200만 편의 학술 논문을 위한 오픈 액세스 아카이브입니다.

본 실습페이지를 통해서 Arxiv.org에서 과학 논문을 로드하여 다운스트림에서 사용할 수 있는 문서 형식으로 변환하는 방법을 알아봅니다.

우선 필요한 패키지들을 설치합니다.

In [ ]:
%pip install --upgrade langchain

# arxiv 라이브러리를 최신 버전으로 업그레이드
%pip install -qU arxiv

# PDF 파일을 텍스트 형식으로 변환하는 PyMuPDF 파이썬 패키지를 설치
%pip install -qU pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 31.9 MB/s eta 0:00:00


query 로 검색할 키워드를 입력하여 해당 키워드가 포함된 문서의 arxid id 와 제목, 서론, url 정보를 가져와서 json 형태로 return 하는 함수를 구현해봅니다.
이때 문서의 양이 방대할 수 있으므로, 문서를 요약까지 이어봅니다.

In [ ]:
# 설치
!pip install -q langchain langchain-google-genai

import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
!apt install libomp-dev
!pip install faiss-cpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached faiss_cpu-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)


In [ ]:
import faiss
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

from langchain.memory import VectorStoreRetrieverMemory

# 모델을 정의합니다.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings, index, InMemoryDocstore({}), {})
# 벡터 조회가 여전히 의미적으로 관련성 있는 정보를 반환한다는 것을 보여주기 위해서 k 값을 1로 정의합니다.
vectorRetriever = vectorstore.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=vectorRetriever)

In [ ]:
from langchain_community.retrievers import ArxivRetriever
import json
import pprint

retriever = ArxivRetriever()

def search_arxiv(query,max_results=10):
    res = {"results": []}
    for result in retriever.arxiv_search(query=query,max_results=max_results).results():
        tmp = {
            "arXiv_id" : result.entry_id.split('abs/')[1][:10],
            "title": result.title,
            "abstract": result.summary,
            "url": result.entry_id}
        res["results"].append(tmp)

    return json.dumps(res)

result = search_arxiv('biogpt', 3) #결과는 최대 3개
pprint.pprint(json.loads(result))


<ipython-input-19-f02582f1270f>:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in retriever.arxiv_search(query=query,max_results=max_results).results():


{'results': [{'abstract': 'Pre-trained language models have attracted '
                          'increasing attention in the\n'
                          'biomedical domain, inspired by their great success '
                          'in the general natural\n'
                          'language domain. Among the two main branches of '
                          'pre-trained language models in\n'
                          'the general language domain, i.e., BERT (and its '
                          'variants) and GPT (and its\n'
                          'variants), the first one has been extensively '
                          'studied in the biomedical domain,\n'
                          'such as BioBERT and PubMedBERT. While they have '
                          'achieved great success on a\n'
                          'variety of discriminative downstream biomedical '
                          'tasks, the lack of generation\n'
                          'ability constrains their a

In [ ]:
# 설치
!pip install -q langchain langchain-google-genai

import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

도큐먼트를 로드하고, 해당 도큐먼트를 번역하고 싶은데, 잘 안되었습니다.. 본 코드를 앞서 arxiv retrieval 과 함께 연결하더라도 마찬가지였습니다.

공부를 깊게 안하고 코드부터 대충 되겠지하고 작성했는데, 역시 그럼 늘 안되네요.
삽질은 여기까지..

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI

# ChatGoogleGenerativeAI 언어 모델을 "gemini-pro" 모델로 초기화합니다.
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever)

questions = [
    "앞서 ArxivRetriever 에게 질의했던 결과를 모두 한글로 해석해줘"
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: 앞서 ArxivRetriever 에게 질의했던 결과를 모두 한글로 해석해줘 

**Answer**: **질의 1:**

**원문:** What is the relationship between the number of parameters in a neural network and its generalization performance?

**번역:** 신경망의 파라미터 수와 일반화 성능 간의 관계는 무엇입니까?

**질의 2:**

**원문:** How can we improve the robustness of neural networks to adversarial examples?

**번역:** 적대적 예제에 대한 신경망의 견고성을 어떻게 향상시킬 수 있습니까?

**질의 3:**

**원문:** What are the challenges and opportunities in using deep learning for natural language processing?

**번역:** 자연어 처리에 딥러닝을 사용하는 데 있어서 어떤 과제와 기회가 있습니까?

**질의 4:**

**원문:** How can we use reinforcement learning to solve complex decision-making problems?

**번역:** 복잡한 의사 결정 문제를 해결하기 위해 강화 학습을 어떻게 사용할 수 있습니까?

**질의 5:**

**원문:** What are the ethical implications of using artificial intelligence in society?

**번역:** 사회에서 인공 지능을 사용하는 것의 윤리적 의미는 무엇입니까? 



ArxivLoader는 다음과 같은 인자를 가집니다:

query: Arxiv에서 문서를 찾는 데 사용되는 자유 텍스트입니다.

load_max_docs(선택): 기본값은 100 입니다. 다운로드할 문서의 수를 제한하는 데 사용합니다. 100개의 문서를 모두 다운로드하는 데는 시간이 걸리므로, 실험을 위해서는 작은 숫자를 사용하도록 합니다.

load_all_available_meta(선택): 기본값은 False 로서 true 가 되는 값들만 다운로드합니다. 기본적으로 가장 중요한 필드인 Published, Title, Authors, Summary 등입니다.

ArxivLoader를 사용하여 arXiv에서 논문을 로드합니다.

In [ ]:
from langchain_community.document_loaders import ArxivLoader
# ArxivLoader를 사용하여 arXiv에서 문서를 로드합니다. query 매개변수는 검색할 논문의 arXiv ID이고, load_max_docs 매개변수는 로드할 최대 문서 수를 지정합니다.
docs = ArxivLoader(query="1605.08386", load_max_docs=2).load()
len(docs)  # 로드된 문서의 개수를 반환합니다.

1

로드된 문서의 수가 결과로 1인 것을 확인하였으니, 배열 index 위치를 활용하여 meta data 를 출력해봅니다.

In [ ]:
docs[0].metadata  # 문서의 메타 정보

{'Published': '2016-05-26',
 'Title': 'Heat-bath random walks with Markov bases',
 'Authors': 'Caprice Stanley, Tobias Windisch',
 'Summary': 'Graphs on lattice points are studied whose edges come from a finite set of\nallowed moves of arbitrary length. We show that the diameter of these graphs on\nfibers of a fixed integer matrix can be bounded from above by a constant. We\nthen study the mixing behaviour of heat-bath random walks on these graphs. We\nalso state explicit conditions on the set of moves so that the heat-bath random\nwalk, a generalization of the Glauber dynamics, is an expander in fixed\ndimension.'}